In [1]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime

## Group Assignment
### Team Number: 14
### Team Member Names: Max Cheng, Nathan Chu, Rahim Merchant, Matthew Yang
### Team Strategy Chosen: Safe

In [2]:
# Constant/Variable Declarations

# Since we must spend all our cash, let's check to make sure it's 0 at the end of our program
capital = 500000

# Final output in Jupyter Notebook
Portfolio_Final = pd.DataFrame()

# DataFrame that our program should write to Stocks_Group_14.csv
Stocks_Final = pd.DataFrame()

In [3]:
'''
- Reads Tickers.csv as a DataFrame
- Turns the first (and only) column into a list
NOTE: Check what the CSV file we will be receiving for the comeptition looks like.
      We will probably have to update this line of code...
'''

ticker_lst = pd.read_csv("Tickers Easy.csv", header=None)[0].tolist()

In [4]:
'''
TODO: Filter the list for invalid or impermissible tickers
- US listed stocks only (so nothing ending in .TO, but we would have to account for other cases too)
'''
american_tickers = ticker_lst

In [9]:
'''
- How do we store the ticker names of the stocks that failed to download?
- The dates at the bottom few rows look like like dividend payments? How do we remove them?
'''

start_date = '2018-01-01'
end_date = '2022-11-16'

data = yf.download(tickers = american_tickers,
                   # threads = True
                   # prepost = True
                   start=start_date,
                   end=end_date).dropna(how='all')
                   
data

[*********************100%***********************]  58 of 58 completed


Adj Close                                                 \
                  AAPL        ABBV         ABT         ACN        AIG   
Date                                                                    
2018-01-02   40.950485   78.019859   54.148964  143.127014  51.953167   
2018-01-03   40.943363   79.240776   54.268684  143.787598  52.529457   
2018-01-04   41.133541   78.788864   54.176586  145.490173  52.739006   
2018-01-05   41.601871   80.160423   54.333176  146.690338  52.957302   
2018-01-08   41.447342   78.876083   54.176586  147.862579  52.608032   
...                ...         ...         ...         ...        ...   
2022-11-09  134.869995  147.619995   99.459999  266.440002  58.459999   
2022-11-10  146.869995  149.720001  104.230003  287.019989  60.590000   
2022-11-11  149.699997  150.160004  104.089996  290.089996  60.389999   
2022-11-14  148.279999  151.740005  102.010002  288.910004  60.240002   
2022-11-15  150.039993  153.039993  103.139999  292.649994  60.540001   

                                                                       ...  \
                  AMZN         AXP          BA        BAC        BIIB  ...   
Date                                                                   ...   
2018-01-02   59.450500   92.141808  282.886383  26.961861  334.170013  ...   
2018-01-03   60.209999   92.709892  283.801270  26.871685  339.850006  ...   
2018-01-04   60.479500   94.251945  282.724396  27.223360  339.989990  ...   
2018-01-05   61.457001   94.466888  294.322296  27.349598  342.489990  ...   
2018-01-08   62.343498   93.597725  295.570770  27.160242  329.649994  ...   
...                ...         ...         ...        ...         ...  ...   
2022-11-09   86.139999  145.860001  168.740005  36.520000  282.309998  ...   
2022-11-10   96.629997  154.750000  177.580002  38.130001  290.700012  ...   
2022-11-11  100.790001  154.889999  177.490005  38.410000  289.450012  ...   
2022-11-14   98.489998  154.130005  173.550003  37.759998  299.059998  ...   
2022-11-15   98.940002  153.889999  175.350006  37.700001  299.779999  ...   

               Volume                                                \
                  SPG           T      TD.TO        TGT         TWX   
Date                                                                  
2018-01-02  1442900.0  42626974.0  2285200.0  8509000.0   5215419.0   
2018-01-03  1849300.0  51851150.0  4851700.0  6057500.0   4033141.0   
2018-01-04  2159500.0  36894187.0  2878800.0  7124500.0   3544977.0   
2018-01-05  2126500.0  29386048.0  3203100.0  5549700.0   5438495.0   
2018-01-08  1874900.0  35276524.0  3887200.0  8337200.0  11114059.0   
...               ...         ...        ...        ...         ...   
2022-11-09  1740100.0  41838400.0  2378800.0  4018400.0         NaN   
2022-11-10  2990900.0  46482700.0  3984800.0  4962900.0         NaN   
2022-11-11  1998400.0  52511000.0  3125300.0  5826900.0         NaN   
2022-11-14  1551600.0  37316500.0  5068700.0  5753800.0         NaN   
2022-11-15  1692200.0  49339300.0  2509800.0  8606000.0         NaN   

                                                                    
                  TXN        UNH        UNP        UPS         USB  
Date                                                                
2018-01-02  4236200.0  3485000.0  4298000.0  5413000.0   5367700.0  
2018-01-03  6918900.0  2417600.0  4653700.0  5655600.0   5043800.0  
2018-01-04  5460400.0  2749100.0  4384900.0  5473100.0   7583600.0  
2018-01-05  5254900.0  2432800.0  5381200.0  3699200.0   6863200.0  
2018-01-08  4549400.0  2850800.0  5472300.0  3243000.0   6817400.0  
...               ...        ...        ...        ...         ...  
2022-11-09  4800400.0  2672600.0  4682700.0  2687100.0   9423000.0  
2022-11-10  9898200.0  3426400.0  4923400.0  3338900.0  14064600.0  
2022-11-11  6911900.0  7032900.0  5176500.0  3851400.0  10692700.0  
2022-11-14  5447300.0  5238400.0  3999000.0  3082500.0   8209800.0  
2022-

In [8]:
'''
TODO: Second filtering iteration
- Remove stocks that failed downloads
- Average MONTHLY volume of at least 200,000 shares from Jan 01, 2022 to Oct 31, 2022
'''

filtered_tickers = american_tickers

In [15]:
prices = pd.DataFrame()
daily_returns = pd.DataFrame()
monthly_returns = pd.DataFrame()

for stock in filtered_tickers:
    ticker_hist = yf.Ticker(stock).history(start=start_date, end=end_date)
    prices[stock] = ticker_hist['Close']
    
    daily_returns = prices.pct_change()
    daily_returns.drop(index=daily_returns.index[0], inplace=True)
    
    monthly_returns=prices.resample('M').ffill().pct_change()
    monthly_returns.drop(index=monthly_returns.index[0], inplace=True)

daily_returns

,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BIIB,...,SPG,T,TD.TO,TGT,TWX,TXN,UNH,UNP,UPS,USB
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-03,-0.000174,0.015649,0.002211,0.004615,0.011092,0.012775,0.006165,0.003234,-0.003345,0.016997,...,-0.004640,-0.023093,0.006241,-0.006802,-0.004461,0.027091,0.010490,0.005597,0.022156,0.009809
2018-01-04,0.004645,-0.005703,-0.001697,0.011841,0.003989,0.004476,0.016633,-0.003794,0.013087,0.000412,...,-0.029250,0.009296,0.008359,-0.019652,0.012350,-0.001291,0.004341,-0.005420,0.006487,0.007698
2018-01-05,0.011386,0.017408,0.002890,0.008249,0.004139,0.016163,0.002281,0.041022,0.004637,0.007353,...,0.009303,0.003684,0.001471,0.010630,0.010472,0.007665,0.019069,0.012740,0.003065,0.006184
2018-01-08,-0.003714,-0.016022,-0.002882,0.007992,-0.006595,0.014425,-0.009200,0.004242,-0.006924,-0.037490,...,0.006542,0.002884,-0.002270,0.009467,-0.005769,0.004766,-0.017357,0.013888,0.012145,0.001446
2018-01-09,-0.000115,0.007538,0.001700,0.003335,0.006307,0.004676,0.003694,0.026697,0.004980,0.020355,...,-0.021979,-0.002607,-0.002698,0.029175,-0.004074,0.006840,0.004983,0.011188,-0.000465,0.006859
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-09,-0.033190,-0.006127,-0.001807,-0.009627,-0.011331,-0.042676,-0.028895,-0.005188,-0.015899,-0.013454,...,-0.021660,-0.010232,-0.009722,-0.053397,0.000000,-0.018559,-0.022403,-0.002941,-0.028319,-0.021055
2022-11-10,0.088975,0.014226,0.047959,0.077241,0.036435,0.121778,0.060949,0.052388,0.044085,0.029719,...,0.063054,0.025027,0.019292,0.074449,0.000000,0.058791,0.006492,0.047097,0.060496,0.037697
2022-11-11,0.019269,0.002939,-0.001343,0.010696,-0.003301,0.043051,0.000905,-0.000507,0.007343,-0.004300,...,0.001333,0.011146,-0.009520,0.054386,0.000000,0.027477,-0.040594,0.021175,0.034065,0.001114


In [16]:
monthly_returns.corr()

,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BIIB,...,SPG,T,TD.TO,TGT,TWX,TXN,UNH,UNP,UPS,USB
AAPL,1.000000,0.205543,0.463910,0.660689,0.172713,0.657856,0.404853,0.319537,0.464648,0.022470,...,0.398339,0.275079,0.390258,0.490045,-0.157350,0.522572,0.352271,0.504700,0.431631,0.297717
ABBV,0.205543,1.000000,0.213112,0.429604,0.464418,0.180643,0.393384,0.351613,0.354150,0.134660,...,0.365225,0.283692,0.366383,0.165786,-0.196195,0.292936,0.427299,0.397431,0.256558,0.273304
ABT,0.463910,0.213112,1.000000,0.580252,0.218221,0.445765,0.306317,0.085859,0.376533,0.144694,...,0.268324,0.376641,0.276689,0.300502,0.010804,0.451210,0.387202,0.555487,0.476531,0.216334
ACN,0.660689,0.429604,0.580252,1.000000,0.507204,0.570521,0.604914,0.484257,0.612188,-0.060830,...,0.577073,0.421948,0.487612,0.541601,-0.054827,0.633727,0.603047,0.725226,0.538045,0.525453
AIG,0.172713,0.464418,0.218221,0.507204,1.000000,0.166233,0.671379,0.555907,0.716632,-0.061174,...,0.636233,0.516065,0.622827,0.267255,0.096589,0.482032,0.305797,0.498169,0.345959,0.723204
AMZN,0.657856,0.180643,0.445765,0.570521,0.166233,1.000000,0.364370,0.185231,0.386729,0.035510,...,0.290290,-0.002391,0.264440,0.298396,-0.065168,0.555997,0.365539,0.515651,0.464197,0.261442
AXP,0.404853,0.393384,0.306317,0.604914,0.671379,0.364370,1.000000,0.662533,0.764705,-0.045409,...,0.672765,0.467242,0.754321,0.437901,0.012735,0.550454,0.396292,0.634238,0.428758,0.704525
BA,0.319537,0.351613,0.085859,0.484257,0.555907,0.185231,0.662533,1.000000,0.592616,-0.076087,...,0.671355,0.382792,0.612981,0.397806,-0.081288,0.531689,0.129124,0.539780,0.336425,0.556866
BAC,0.464648,0.354150,0.376533,0.612188,0.716632,0.386729,0.764705,0.592616,1.000000,0.097519,...,0.722575,0.543087,0.754119,0.346026,-0.042670,0.656814,0.443719,0.658332,0.493894,0.872492
BIIB,0.022470,0.134660,0.144694,-0.060830,-0.061174,0.035510,-0.045409,-0.076087,0.097519,1.000000,...,-0.035318,0.044706,0.089823,0.030224,0.107492,0.055838,0.175085,0.042516,0.055390,-0.020932


In [ ]:
def find_min(monthly_returns):
    

In [ ]:
'''
TODO: Calculations
Let's discuss how we want to approach this from a finance POV before committing to deep into anything

Some ideas...
1. Fetch beta for each stock, find a combination of 25 that sum to a number close to 0
- Pros: Seems easy enough (famous last words)
- Cons: Everything is becoming more correlated now, we may have to look at individual correlations

2. Create a correlation matrix (Correlation of each stock with each other stock), then pick the
   PAIRS that sum to the lowest number. Ideally, we would have a portfolio of pairs of stocks,
   with the 2 stocks in each pair having perfect NEGATIVE correlation. Thus the portfolio never
   fluctuates.
'''

In [ ]:
'''
Bonus: Can we check to see if any companies are reporting their quarterly earnings over the week
of the competition and try avoid those companies if possible?
'''

In [38]:
# Write to CSV
Stocks_Final.to_csv("Stocks_Group_14.csv")

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Max Cheng, Nathan Chu, Rahim Merchant, Matthew Yang